<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/RNA_Workshop/Explore_Nell2_Pathways.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scanpy --quiet
!pip install pybiomart --quiet
!pip install python-igraph --quiet
!pip install louvain --quiet
!pip install pynndescent --quiet


!pip install scipy

In [ ]:
import h5py
import numpy as np
from matplotlib import pyplot as plt
import scanpy as sc
import tarfile
import os
import anndata as ad
import pandas as pd
import pybiomart
from tqdm import tqdm
import urllib.request
from IPython.display import clear_output
from matplotlib.pyplot import rc_context
import scipy
import logging
import itertools

In [ ]:
!pip install goatools

In [ ]:
import goatools

In [ ]:
### quick import
import urllib
import json
from matplotlib import rcParams
def import_mpl_config(FS=6):
    _ = urllib.request.urlretrieve('https://github.com/dtabuena/Resources/raw/38d1acaa72ac6d0c1893872ed092f0a8a2bd8d35/Matplotlib_Config/mpl_config_FS6.json','mpl_config_FS6.json')
    _ = urllib.request.urlretrieve('https://github.com/dtabuena/Resources/raw/38d1acaa72ac6d0c1893872ed092f0a8a2bd8d35/Matplotlib_Config/mpl_config_FS8.json','mpl_config_FS8.json')
    with open(f"./mpl_config_FS{FS}.json",'r') as import_file:
        fig_config = json.load(import_file)
    rcParams.update(fig_config)
    return fig_config
_ = import_mpl_config()

In [ ]:
def init_GO():
    # !pip install goatools --quiet


    '''Get Gene Lists and metadata from ncbi'''
    import os
    import urllib.request
    gene_list_url='https://github.com/dtabuena/Resources/raw/1e3f0ef18ba127b71a9e6b93f7624e3a28fe87c1/GO%20Files/gene_result.txt'
    urllib.request.urlretrieve(gene_list_url, 'gene_result.txt')
    scripts_path = [p for p in os.environ['PATH'].split(';') if 'Scripts' in p][0]
    ncbi_path = os.path.join(scripts_path,'ncbi_gene_results_to_python.py')
    !python $ncbi_path -o genes_ncbi_mus_musculus_proteincoding.py gene_result.txt
    from genes_ncbi_mus_musculus_proteincoding import GENEID2NT as GeneID2nt_mus



    '''Get Key Funcs'''
    from goatools.base import download_go_basic_obo
    from goatools.base import download_ncbi_associations
    from goatools.obo_parser import GODag
    from goatools.anno.genetogo_reader import Gene2GoReader
    from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS

    '''Download Current Go Annotations'''
    obo_fname = download_go_basic_obo()
    fin_gene2go = download_ncbi_associations()
    obodag = GODag("go-basic.obo")




    '''Get Mapper from Symbol to Gene and Inv'''
    mapper = {}
    for key in GeneID2nt_mus:
        mapper[GeneID2nt_mus[key].Symbol] = GeneID2nt_mus[key].GeneID
    inv_map = {v: k for k, v in mapper.items()}



    '''Read NCBI's gene2go. Store annotations in a list of namedtuples '''
    objanno = Gene2GoReader(fin_gene2go, taxids=[10090])
    # Get namespace2association where:
    #    namespace is:
    #        BP: biological_process
    #        MF: molecular_function
    #        CC: cellular_component
    #    assocation is a dict:
    #        key: NCBI GeneID
    #        value: A set of GO IDs associated with that gene
    ns2assoc = objanno.get_ns2assc()


    '''Create a GO Object'''
    goeaobj = GOEnrichmentStudyNS(
            GeneID2nt_mus.keys(), # List of mouse protein-coding genes
            ns2assoc, # geneid/GO associations
            obodag, # Ontologies
            propagate_counts = False,
            alpha = 0.05, # default significance cut-off
            methods = ['fdr_bh']) # defult multipletest correction method


    ''' PASS '''
    num_terms = {}
    GO_items = []
    temp = goeaobj.ns2objgoea['BP'].assoc
    num_terms['biological_process']=len(temp)
    for item in temp:
        GO_items += temp[item]
    temp = goeaobj.ns2objgoea['CC'].assoc
    num_terms['cellular_component']=len(temp)
    for item in temp:
        GO_items += temp[item]
    temp = goeaobj.ns2objgoea['MF'].assoc
    num_terms['molecular_function']=len(temp)
    for item in temp:
        GO_items += temp[item]



    def go_it(test_genes):
        ''' Quick Access Function for doing the GO associations '''
        logging.info(f'input genes: {len(test_genes)}')
        mapped_genes = []
        for gene in test_genes:
            try:
                mapped_genes.append(mapper[gene])
            except:
                pass
        logging.info(f'mapped genes: {len(mapped_genes)}')
        goea_results_all = goeaobj.run_study(mapped_genes)
        goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
        GO = pd.DataFrame(list(map(lambda x: [x.GO, x.goterm.name, x.goterm.namespace, num_terms[x.goterm.namespace], x.p_uncorrected, x.p_fdr_bh,\
                    x.ratio_in_study[0], x.ratio_in_study[1], GO_items.count(x.GO), list(map(lambda y: inv_map[y], x.study_items)),\
                    ], goea_results_sig)), columns = ['GO', 'term', 'class', 'class_size', 'p', 'p_corr', 'n_genes',\
                                                        'n_study', 'n_go', 'study_genes'])
        GO = GO[GO.n_genes > 1]
        GO['enrich_ratio'] = ( GO['n_genes']/GO['n_study'] ) / ( GO['n_go']/GO['class_size'] )
        return GO


    all_GO = None
    goea_results_all = goeaobj.run_study([mapper['Apoe']])
    all_GO = pd.DataFrame(list(map(lambda x: [x.GO, x.goterm.name, x.goterm.namespace, num_terms[x.goterm.namespace], GO_items.count(x.GO),
                        ], goea_results_all)), columns = ['GO', 'term', 'class', 'class_size', 'n_go',])


    return go_it, num_terms ,goeaobj,goea_results_all,all_GO

os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/_Nell2_enrichment/")
go_it, num_terms, goeaobj,goea_results_all,all_GO = init_GO()

In [ ]:
def trim_key(k):
    floxed_dict = {'GSM5106175_YH_KZ03_01':('E3fKI_Syn_Cre602_15m','GSM5106175_602_E3fKI_15_XX'),
                   'GSM5106176_YH_KZ03_03':('E4fKI_Syn_Cre475_15m','GSM5106176_475_E4fKI_15_XX')}
    for f in floxed_dict.keys():
        if f in k: return floxed_dict[f][1]
    k = k.replace('_raw_gene_bc_matrices_h5.h5',"")
    return k
def query_capitilaziation(gene,adata):
    try:
        return adata.var.index[ [g.lower() for g in list(adata.var.index)].index(gene.lower()) ]
    except:
        return gene + ' not_found'
def z_score(x,axis=-1):
    x=np.array(x)
    return (x-np.mean(x,axis=axis))/np.std(x,axis=axis)



In [ ]:
os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Seurat_Scanpy/Scanpy_data/")
adata = sc.read_h5ad('./2023_11_07_KZ_anndata.h5ad')

In [ ]:
display(adata.obs.head())

meta_df = pd.read_csv('./kz_metadata.csv').set_index('Barcodes')

adata_meta= adata.copy()
adata_meta.obs["Cluster_ID"]=np.nan
adata_meta.obs["Cluster_ID"]= meta_df["Cluster_ID"]
display(adata_meta.obs.head())

all_cats = list(set(meta_df["Cluster_ID"]))
print(all_cats)

In [ ]:
########### Sub Divide Clusters of Interest
os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/_Nell2_enrichment/")
# dgc_01_adata =  adata_meta[adata_meta.obs["Cluster_ID"] == '01 Dentate Gyrus Granule Cells'].copy()
# display(dgc_01_adata)
dgc_02_adata =  adata_meta[adata_meta.obs["Cluster_ID"] == '02 Dentate Gyrus Granule Cells'].copy()
display(dgc_02_adata)
CA3_06_adata =  adata_meta[adata_meta.obs["Cluster_ID"] == '06 CA2/CA3 Pyramids'].copy()
display(CA3_06_adata)


In [ ]:
import statsmodels

In [ ]:
def find_corr_genes(adata,gene,pct=1,to_plot=True):
    X_ = adata.X.toarray()
    gene_ind = list(adata.var['name']).index(gene)
    target_row = X_[:,gene_ind]
    num_genes= X_.shape[1]
    gene_corr = np.ones([num_genes])
    p_vals = np.zeros([num_genes])
    for g in range(num_genes):
        gene_corr[g],p_vals[g] = scipy.stats.pearsonr( target_row,X_[:,g])


    gene_corr[np.isnan(gene_corr)] = 0
    gene_rank =np.argsort(np.argsort(-gene_corr))
    gene_corr_plot=gene_corr.copy()
    gene_corr_plot[gene_ind]=np.nan

    low,high = np.nanpercentile(gene_corr,[pct,100-pct])

    high_bool = gene_corr>high
    low_bool = gene_corr<low

    high_names=adata.var['name'][high_bool]
    low_names=adata.var['name'][low_bool]

    FDR_p_vals = p_vals.copy()
    p_not_nan=np.logical_not(np.isnan(p_vals))
    FDR_bool = np.zeros_like(FDR_p_vals,dtype=bool)
    FDR_bool[p_not_nan], FDR_p_vals[p_not_nan] = statsmodels.stats.multitest.fdrcorrection(p_vals[p_not_nan], alpha=0.05, method='indep', is_sorted=False)
    if to_plot:
        fig,ax=plt.subplots(1,2,figsize=(3,1),width_ratios=(2,2),dpi=300)
            # fig,ax=plt.subplots(1,1,figsize=(3,1),dpi=300)
            # ax=[ax]
        ax[0].scatter(gene_rank/num_genes*100,gene_corr_plot,s=1,color='k')
        ax[0].scatter((gene_rank/num_genes*100)[FDR_bool],gene_corr_plot[FDR_bool],s=1,color='r')

        ax[0].axhline(high,color='k',linewidth=1)
        ax[0].axhline(low,color='k',linewidth=1)

        ax[0].axhline(high,color='k',linewidth=1)
        ax[0].axhline(low,color='k',linewidth=1)

        ax[0].set_xlabel('Percentile Rank')
        ax[0].set_ylabel('Pearson R')

        ax[1].scatter(gene_corr,-np.log10(FDR_p_vals),s=1,color='k')
        ax[1].scatter(gene_corr[FDR_bool],-np.log10(FDR_p_vals)[FDR_bool],s=1,color='r')


    plt.show()

    results_dict = {'gene_corr':gene_corr, 'high_bool':high_bool,'low_bool':low_bool ,'high_names':high_names,'low_names':low_names,'gene_rank':gene_rank}
    results_dict['table']= pd.DataFrame({'gene_name':adata.var['name'],
                                         'gene_rank':gene_rank,
                                         'gene_corr':gene_corr,
                                         'p_vals':p_vals,
                                         'FDR_p_vals':FDR_p_vals,
                                         'high_bool':high_bool,
                                         'low_bool':low_bool}).set_index('gene_name').sort_values('gene_rank',axis=0)

    return results_dict, fig, ax

pct = 5
dgc_02_nell_results,_,_ = find_corr_genes(dgc_02_adata,'Nell2',pct=pct)
dgc_02_apoe_results,_,_ = find_corr_genes(dgc_02_adata,'Apoe',pct=pct)

CA3_06_nell_results,_,_ = find_corr_genes(CA3_06_adata,'Nell2',pct=pct)
CA3_06_apoe_results,_,_ = find_corr_genes(CA3_06_adata,'Apoe',pct=pct)


In [ ]:

def write_tables(res,prefix):
    high_table = res['table']
    high_table = high_table[high_table['high_bool']]
    high_table.to_csv(prefix+'_high_corr_names.csv')

    low_table = CA3_06_nell_results['table']
    low_table = high_table[low_table['low_bool']]
    low_table.to_csv(prefix+'_low_corr_names.csv')
    return None

os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/_Nell2_enrichment/")
write_tables(CA3_06_nell_results,'CA3_06_nell')
write_tables(CA3_06_apoe_results,'CA3_06_apoe')
write_tables(dgc_02_nell_results,'dgc_02_nell')
write_tables(dgc_02_apoe_results,'dgc_02_apoe')



In [ ]:
logging.basicConfig(filename='example.log',level=logging.DEBUG)
def get_onto_from_dict(dict_results):
    h = len(dict_results['high_names'])
    l = len(dict_results['low_names'])
    logging.info(f"{h} high genes and {l} low genes")
    dict_results['go_high'] = go_it(dict_results['high_names'])
    dict_results['go_low'] = go_it(dict_results['low_names'])
    return dict_results


CA3_06_nell_results = get_onto_from_dict(CA3_06_nell_results)
CA3_06_apoe_results = get_onto_from_dict(CA3_06_apoe_results)
dgc_02_nell_results = get_onto_from_dict(dgc_02_nell_results)
dgc_02_apoe_results = get_onto_from_dict(dgc_02_apoe_results)



In [ ]:
go_group_dict={'CA3_nell_h':CA3_06_nell_results['go_high'],
               'CA3_apoe_h':CA3_06_apoe_results['go_high'],
               'dgc_nell_h':dgc_02_nell_results['go_high'],
               'dgc_apoe_h':dgc_02_apoe_results['go_high'],
               }
for k,v in go_group_dict.items():
    in_bio = ['biological_process' in c for c in v['class']]
    go_group_dict[k] = list(v['GO']) #
    go_group_dict[k] = list(v['GO'][in_bio])


In [ ]:
go_group_dict[k]

In [ ]:
def membership_dict_to_df(group_dict: dict) -> pd.DataFrame:
    """
    get dict with keys:group and values:list of items.
    convert dict to a pd.DataFrame with index:items and
    columns for membership in label. A numerical embeding
    of membership overlaps is created for simplicity after.
    A look up dict its returned to interpred embeddings
    """

    inicies = [x for v in group_dict.values() for x in v]
    inicies = list(set(inicies))
    mebership_df = pd.DataFrame(index=inicies,columns=list(group_dict.keys()))
    mebership_df['emb_combo'] = np.nan
    possible_combos = list(itertools.product([True,False], repeat=len(group_dict.keys())))
    combo_sum = [np.sum(c) for c in possible_combos]
    possible_combos = [possible_combos[i] for i in np.argsort(combo_sum)]
    embed_combos = {c:i for i,c in enumerate(possible_combos)}
    for r in mebership_df.index:
        row_bool = [ r in vals for vals in group_dict.values()]
        mebership_df.loc[r,list(group_dict.keys())] = row_bool
        mebership_df.loc[r,'emb_combo'] = embed_combos[tuple(row_bool)]
    return mebership_df, embed_combos


In [ ]:
import json
FS = 6
fig_config = {'font.size': FS, 'font.family': 'arial','axes.linewidth':.5,
              'xtick.major.width': 0.5,'ytick.major.width': 0.5,
              'figure.titlesize':FS,'axes.titlesize': FS,'xtick.labelsize': FS,
              'ytick.labelsize': FS,'axes.labelsize': FS,'legend.fontsize': FS,
              'savefig.dpi': 300,'figure.dpi': 300}
with open(f"./fig_congig_FS{FS}.json",'w') as outfile:
    outfile.write( json.dumps(fig_config, indent=4) )



In [ ]:

def set_font_all(FS):
    plt.rcParams.update({'font.size': FS, 'font.family': 'arial','axes.linewidth':.5,
                         'xtick.major.width': 0.5,'ytick.major.width': 0.5,
                         'figure.titlesize':FS,'axes.titlesize': FS,'xtick.labelsize': FS,
                         'ytick.labelsize': FS,'axes.labelsize': FS,'legend.fontsize': FS})
    return None
set_font_all(6)

In [ ]:
def upset_plot(group_dict, figsize=(3,2), exclude_all_none=True,write_name=None):

    'group_dict->dict with keys=categories/groups and values=list(members)'

    mebership_df, embed_combos = membership_dict_to_df(go_group_dict)
    possible_combos = list(embed_combos.keys())
    if exclude_all_none: #exclude members that do not intersect any group
        possible_combos = [c for c in possible_combos if np.array(c).any()]
        false_tupple = tuple(np.full((len(possible_combos[0])),False))
        del embed_combos[false_tupple]

    fig,ax=plt.subplots(2,2,figsize=figsize,width_ratios=[3,.5],height_ratios=[3,1],dpi=300)
    null_ax = ax[0,1]
    null_ax.axis('off')
    combo_ax = ax[1,0]
    overlap_ax = ax[0,0]
    set_size_ax = ax[1,1]

    group_names = [c for c in mebership_df.columns if str(c) not in 'emb_combo']


    """Draw Dots and Connect"""
    true_xy = np.where(possible_combos)
    false_xy = np.where(np.logical_not(possible_combos))
    dot_size=12
    combo_ax.scatter(true_xy[0],true_xy[1],color='k',s=dot_size)
    combo_ax.scatter(false_xy[0],false_xy[1],color='lightgrey',s=dot_size)
    combo_ax.set_yticks(range(4),group_names)
    combo_ax.set_xticks([])
    combo_ax.set_ylim([-.5,len(group_names)-.5])
    my_map = np.cumsum(np.ones_like(possible_combos),axis=1)*possible_combos
    for row,vals in enumerate(my_map):
        vals_nz = [v for v in vals if v > 0]
        if np.sum(vals>0)>1:
            combo_ax.plot([row,row],[np.min(vals_nz)-1,np.max(vals_nz)-1],color='k',linewidth=.5 )



    """Plot Overlap Bars"""
    intersection_counts, bin_edges = np.histogram(mebership_df['emb_combo'],
                                                  bins=len(possible_combos),
                                                  range=[np.min(mebership_df['emb_combo'])-.5,np.max(mebership_df['emb_combo'])+.5]) #
    overlap_ax.bar(bin_edges[:-1]-.5,intersection_counts,color='k')
    overlap_ax.set_xlim(combo_ax.get_xlim() )
    overlap_ax.set_xticks([])
    overlap_ax.set_ylabel('Intersection (#)')


    """Plot Groups Sizes"""
    set_size_ax.barh( list(group_dict.keys()), [len(v) for v in group_dict.values()],color='k'   )
    set_size_ax.set_yticks([])
    set_size_ax.set_xlabel('Group Size (#)')

    from matplotlib import rcParams
    rcParams.update({'figure.autolayout': True})

    if write_name is not None:
        fig.savefig(f'./{write_name}.svg',format='svg',dpi=300,bbox_inches="tight")
    return (fig,ax), mebership_df, embed_combos


fig_ax, mebership_df,embed_combos = upset_plot(go_group_dict,write_name='High_Correlation_GO_Terms')

df = all_GO.copy().set_index('GO')
df = df[ [go in mebership_df.index.to_list() for go in df.index] ]
mebership_df = pd.concat([mebership_df, df], axis=1)


mebership_df.to_csv('./nell_apoe_CA_DG_go_term_intersections.csv')

In [ ]:
import matplotlib as mpl

In [ ]:

go_df_dict =  {'CA3_nell_h':CA3_06_nell_results['go_high'].set_index('GO'),
               'CA3_apoe_h':CA3_06_apoe_results['go_high'].set_index('GO'),
               'dgc_nell_h':dgc_02_nell_results['go_high'].set_index('GO'),
               'dgc_apoe_h':dgc_02_apoe_results['go_high'].set_index('GO'),
               }

go_df_dict['CA3_nell_h']
for k,v in go_df_dict.items():
    in_bio = ['biological_process' in c for c in v['class']]
    go_df_dict[k] = v[in_bio]




go_df_dict['CA3_nell_h']['rank'] = np.argsort(np.argsort(go_df_dict['CA3_nell_h']['p_corr']))+1
go_df_dict['CA3_nell_h'].sort_values(by='p_corr',ascending=True)

go_df_dict['dgc_nell_h']['rank'] = np.argsort(np.argsort(go_df_dict['dgc_nell_h']['p_corr']))+1
go_df_dict['dgc_nell_h'].sort_values(by='p_corr',ascending=True)

select_gos = mebership_df[mebership_df['emb_combo']==6].index.to_list()
rank_pairs = [  (go_df_dict['dgc_nell_h'].loc[g,'rank'] ,  go_df_dict['CA3_nell_h'].loc[g,'rank']) for g in select_gos  ]
z_ranks = [ x**2+y**2 for (x,y) in rank_pairs]


select_gos = list(np.array(select_gos)[np.argsort(np.argsort(z_ranks))<10])


cmap = mpl.colormaps.get_cmap('hot')



vmin = 10
vmax = 20
vmin=-np.log10(.05)


fig = plt.figure(layout=None ,figsize=(2,2),dpi=300)
gs = fig.add_gridspec(nrows=2, ncols=3,width_ratios=[1,1,.3])
ax = [fig.add_subplot(gs[:2, 0]),
      fig.add_subplot(gs[:2, 1]),
      fig.add_subplot(gs[0, 2]),
      fig.add_subplot(gs[1, 2])]


for i,g in enumerate(select_gos):
    x = go_df_dict['CA3_nell_h'].loc[g,'enrich_ratio']
    y = i
    size = go_df_dict['CA3_nell_h'].loc[g,'n_genes']
    color_level = -np.log10(go_df_dict['CA3_nell_h'].loc[g,'p_corr'])
    ax[0].scatter(x,y,s=size,c=color_level,cmap=cmap,vmin=vmin,vmax=vmax)
    ax[0].set_title('CA3')

    x = go_df_dict['dgc_nell_h'].loc[g,'enrich_ratio']
    y = i
    size = go_df_dict['dgc_nell_h'].loc[g,'n_genes']
    color_level = -np.log10(go_df_dict['dgc_nell_h'].loc[g,'p_corr'])
    cbr = ax[1].scatter(x,y,s=size,c=color_level,cmap=cmap,vmin=vmin,vmax=vmax)
    ax[1].set_title('DGC')

terms = [go_df_dict['CA3_nell_h'].loc[g,'term'] for g in select_gos]
ax[0].set_yticks(range(len(terms)), terms)
ax[0].set_ylabel('Gene Ontology Terms')
ax[1].set_yticks([])


plt.colorbar(cbr,cax=ax[3])
ax[3].set_ylabel('-log10(p-value)')


sizes = [5,10,20,50]
n=len(sizes)
ax[2].scatter(np.ones(n),np.linspace(0,1,n,endpoint=False),s=sizes,color='k')
ax[2].text(1.2,.5,'Gene #',rotation=90,ha='center',va='center')
ax[2].axis('off')
for i,s in enumerate(sizes):
    ax[2].text(1,i/n,f"     {s}",ha='left',va='center')
    print(i/n)


ax[2].set_ylim(-.2,1.2)

for a in ax[:2]:
    a.set_xlim(1,7)
    a.set_ylim(-.5,len(select_gos)-.5)
    a.set_xlabel('Fold Enrichment')


plt.tight_layout()

In [ ]:
[i,j for (i,j) in enumerate(terms)]

In [ ]:




go_df_dict =  {'CA3_nell_h':CA3_06_nell_results['go_high'].set_index('GO'),
               'CA3_apoe_h':CA3_06_apoe_results['go_high'].set_index('GO'),
               'dgc_nell_h':dgc_02_nell_results['go_high'].set_index('GO'),
               'dgc_apoe_h':dgc_02_apoe_results['go_high'].set_index('GO'),
               }


# select_gos = mebership_df[mebership_df['emb_combo']==6].index.to_list()


go_df_dict_selct = {k:v[  [g in select_gos for g in v.index]  ] for (k,v) in go_df_dict.items()}
# for k,v in go_df_dict_selct.items():
#     display(v.sort_values(by='enrich_ratio',ascending=False))

go_df_dict['CA3_nell_h']['rank'] = np.argsort(np.argsort(go_df_dict['CA3_nell_h']['p_corr']))
go_df_dict['CA3_nell_h'].sort_values(by='p_corr',ascending=True)

go_df_dict['dgc_nell_h']['rank'] = np.argsort(np.argsort(go_df_dict['dgc_nell_h']['p_corr']))
go_df_dict['dgc_nell_h'].sort_values(by='p_corr',ascending=True)

fig,ax=plt.subplots(1,figsize=(4,3),dpi=300)
for g in select_gos:
    x = go_df_dict['CA3_nell_h'].loc[g,'p_corr']
    y = go_df_dict['dgc_nell_h'].loc[g,'p_corr']


    width = go_df_dict['CA3_nell_h'].loc[g,'enrich_ratio']
    height = go_df_dict['dgc_nell_h'].loc[g,'enrich_ratio']


    x=-np.log10(x)
    y=-np.log10(y)

    f = .3
    width=width*f
    height=height*f

    #
    ax.scatter(x,y,s=0,c='w',alpha=1)
    e=mpl.patches.Ellipse(   (x,y),width=width,height=height )
    ax.add_artist(e)
    e.set_alpha(.5)
    # e.set_facecolor('k')
    # e.set_clip_box(ax.bbox)

    if x>6 and y>6:
        label = go_df_dict['dgc_nell_h'].loc[g,'term']
        ax.text(x,y,label  )

ax.set_xlim(0,24)
ax.set_ylim(0,18)

scale_elip=[mpl.patches.Ellipse( (1,12),width=1*f,height=1*f ),
            mpl.patches.Ellipse( (1,14),width=3*f,height=3*f ),
            mpl.patches.Ellipse( (1,16),width=5*f,height=5*f )]
for e in scale_elip:
    ax.add_artist(e)
    e.set_alpha(.5)
    e.set_facecolor('k')

print(x,y,wid,height)
# plt.show()
# plt.tight_layout()
